In [1]:
# importing the libraries
import requests 
import pandas as pd
import json
import seaborn as sns
import streamlit as st
from datetime import date
pd.set_option('display.float_format', '{:.11f}'.format)
%matplotlib inline

<h2>Creating historic prices dataframe</h2>

In [2]:
today = date.today()
today = today.strftime('%Y-%m-%d')

In [3]:
coins = ['USDC','ETH','BNB','XRP','BUSD','ADA','SOL','MATIC','DOT','SHIB','TRX','DAI','AVAX','UNI',
        'ATOM','LTC','LINK','XLM','CRO','XMR','ALGO','BCH','VET','LUNC','FIL','XTZ','AAVE', 'THETA','EOS','HT','BSV',
        'USDP','TUSD','MKR','ETHW','BTT','ZEC','NEO','NEXO','PAXG','RUNE','DASH','ENJ','USTC','LUNA','CEL','XEM',
        'COMP','YFI','ONE','OMG','ONT','EURS']

In [4]:
def jsonreader(from_currency,to_currency,since,until):
    parametr = {
    'from':from_currency,
    'to':to_currency,
    'since':since,
    'until':until,
    'limit': 400,
    'period': 'D1'}
    try:
        response = requests.get('https://api.exchange.cryptomkt.com/api/3/public/price/history', params=parametr)
        price = response.json()
        for i in price:
            from_ = i
            d = price[i]
            for key, values in d.items():
                if type(values) == list:
                    for k,v in values[0].items():
                        data = pd.DataFrame.from_dict(values)
                        break
                else:
                    to = values
        data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S')
        df_list = ['open','close','min','max']
        for i in df_list:
            data[i] = pd.to_numeric(data[i], errors='coerce')
        currencies = {'from_currency': from_, 'to_currency': to}
        data = data.assign(**currencies)
        print("Status code: ",response.status_code," ",from_)
    except:
        print("No Data for: ",from_currency)
        pass
    return data

In [5]:
currencies = jsonreader('BTC','USDT','2022-01-01',today)

Status code:  200   BTC


In [6]:
for i in coins:
    try:
        currencies = currencies.append(jsonreader(i,'USDT','2022-01-01',today))
    except:
        pass

Status code:  200   USDC
Status code:  200   ETH
Status code:  200   BNB
Status code:  200   XRP
Status code:  200   BUSD
Status code:  200   ADA
Status code:  200   SOL
Status code:  200   MATIC
Status code:  200   DOT
Status code:  200   SHIB
Status code:  200   TRX
Status code:  200   DAI
Status code:  200   AVAX
Status code:  200   UNI
Status code:  200   ATOM
Status code:  200   LTC
Status code:  200   LINK
Status code:  200   XLM
Status code:  200   CRO
Status code:  200   XMR
Status code:  200   ALGO
Status code:  200   BCH
Status code:  200   VET
Status code:  200   LUNC
Status code:  200   FIL
Status code:  200   XTZ
Status code:  200   AAVE
Status code:  200   THETA
Status code:  200   EOS
Status code:  200   HT
Status code:  200   BSV
Status code:  200   USDP
Status code:  200   TUSD
Status code:  200   MKR
Status code:  200   ETHW
Status code:  200   BTT
Status code:  200   ZEC
Status code:  200   NEO
Status code:  200   NEXO
Status code:  200   PAXG
Status code:  200   RUN

In [7]:
currencies.reset_index(drop=True, inplace=True)

In [8]:
currencies.head()

,timestamp,open,close,min,max,from_currency,to_currency
0,2022-12-31 00:00:00+00:00,16605.86000000000,16540.27000000000,16478.00000000000,16634.76000000000,BTC,USDT
1,2022-12-30 00:00:00+00:00,16634.79000000000,16605.86000000000,16338.81000000000,16655.00000000000,BTC,USDT
2,2022-12-29 00:00:00+00:00,16540.67000000000,16638.14000000000,16488.45000000000,16664.49000000000,BTC,USDT
3,2022-12-28 00:00:00+00:00,16705.62000000000,16543.92000000000,16470.03000000000,16775.51000000000,BTC,USDT
4,2022-12-27 00:00:00+00:00,16920.46000000000,16706.34000000000,16600.00000000000,16968.63000000000,BTC,USDT


<h2>Extracting Crypto wallet balance </h2>

In [9]:
with open('/Users/brauliocastelblancocastro/Desktop/CryptoMKT/public_key.txt')as f:
    public_key = f.readlines()
with open('/Users/brauliocastelblancocastro/Desktop/CryptoMKT/private_key.txt')as f:
    private_key = f.readlines()
url_balance = 'https://api.exchange.cryptomkt.com/api/3/wallet/balance'
authentication = (public_key[0],private_key[0])

In [10]:
wallet_response = requests.get(url_balance,auth=authentication)
wallet_response.status_code

200

In [11]:
balance = pd.DataFrame.from_records(wallet_response.json())

In [12]:
balance

,currency,available,reserved
0,AAVE,0.0751892,0
1,ALGO,0.702151,0
2,ATOM,0.650883,0
3,BNB,0.01870182,0
4,BTC,0.00662766,0
5,CLP,0.63,0
6,ENJ,6.96332244,0
7,ETH,0.032094638677,0
8,MATIC,4,0
9,DOT,0.00014499,0


<h2>Extracting transactions</h2>

In [13]:
transactions_url = "https://api.exchange.cryptomkt.com/api/3/wallet/transactions"
transactions_response = requests.get(transactions_url, auth=authentication)
transactions_response.status_code

200

In [14]:
transactions_history = pd.DataFrame.from_records(transactions_response.json())
transactions_history.head()

,id,created_at,updated_at,status,type,subtype,native
0,212567174,2022-12-29T15:37:51.445Z,2022-12-30T17:56:56.445Z,SUCCESS,WITHDRAW,FIAT,{'tx_id': '7eb11018-0d7e-484d-940e-1fbc4e045bb...
1,212567104,2022-12-29T15:36:51.811Z,2022-12-29T15:36:51.865Z,SUCCESS,DEPOSIT,OFFCHAIN,{'tx_id': 'd24dc001-5dc3-4579-b383-8fbf6a71a80...
2,212567088,2022-12-29T15:36:41.392Z,2022-12-29T15:36:41.438Z,SUCCESS,WITHDRAW,OFFCHAIN,{'tx_id': 'e9efb541-f93d-4ffa-b0c2-d5d32f7d97b...
3,212189863,2022-12-28T14:01:59.288Z,2022-12-28T14:01:59.362Z,SUCCESS,DEPOSIT,OFFCHAIN,{'tx_id': '59b6af52-fbe0-4979-8788-b39710e9d78...
4,208980372,2022-12-19T16:52:04.517Z,2022-12-19T16:52:04.569Z,SUCCESS,WITHDRAW,OFFCHAIN,{'tx_id': 'bdb610e1-6e2a-466f-9e97-73dc3209803...


In [15]:
new_native = pd.DataFrame()
for key, values in transactions_history['native'].items():
        new_native = new_native.append(values, ignore_index=True)
transactions_history.drop(columns='native', inplace=True)
transactions_history = transactions_history.join(new_native)
transactions_history.tail(10)

,id,created_at,updated_at,status,type,subtype,amount,currency,index,tx_id,address,confirmations,hash,senders
67,160964607,2021-11-02T03:01:14.737Z,2021-11-02T03:01:15.103Z,SUCCESS,DEPOSIT,FIAT,100000,CLP,8846353552.00000000000,ba89d905-0e79-40ea-8cd2-22e2f2cff4b9,NaN,NaN,NaN,NaN
68,159085068,2021-10-24T13:15:35.014Z,2021-10-24T13:15:35.027Z,SUCCESS,DEPOSIT,OFFCHAIN,0.02504187,BNB,8789324594.00000000000,0077a1bc-8021-4f03-b975-2806da0d88a0,NaN,NaN,NaN,NaN
69,159085054,2021-10-24T13:15:32.811Z,2021-10-24T13:15:32.854Z,SUCCESS,WITHDRAW,OFFCHAIN,10000,CLP,8789324327.00000000000,9841650c-536f-4582-be4e-efb096c236e4,NaN,NaN,NaN,NaN
70,159084967,2021-10-24T13:14:54.174Z,2021-10-24T13:14:54.215Z,SUCCESS,DEPOSIT,OFFCHAIN,0.001476114235,ETH,8789321498.00000000000,c01c2cba-a68b-478d-a298-aad223bf3261,NaN,NaN,NaN,NaN
71,159084964,2021-10-24T13:14:51.81Z,2021-10-24T13:14:51.841Z,SUCCESS,WITHDRAW,OFFCHAIN,5000,CLP,8789321305.00000000000,fe12aaef-413d-413b-88a4-f391e9fc7df0,NaN,NaN,NaN,NaN
72,159084876,2021-10-24T13:14:14.592Z,2021-10-24T13:14:14.659Z,SUCCESS,DEPOSIT,OFFCHAIN,0.00019766,BTC,8789318501.00000000000,059dd9b6-8723-4358-ae87-9ed70e9a7939,NaN,NaN,NaN,NaN
73,159084869,2021-10-24T13:14:10.942Z,2021-10-24T13:14:11.009Z,SUCCESS,WITHDRAW,OFFCHAIN,10000,CLP,8789318188.00000000000,57a7c0e7-b94c-474d-9914-d0a4cbb69240,NaN,NaN,NaN,NaN
74,158624998,2021-10-21T20:03:57.905Z,2021-10-21T20:03:57.944Z,SUCCESS,DEPOSIT,OFFCHAIN,0.001493117632,ETH,8771133094.00000000000,667b2407-07c2-4e9e-95db-af3e0cb4b801,NaN,NaN,NaN,NaN
75,158624983,2021-10-21T20:03:54.691Z,2021-10-21T20:03:54.777Z,SUCCESS,WITHDRAW,OFFCHAIN,5000,CLP,8771132707.00000000000,05466cf3-df00-4f05-93c8-152a78892f55,NaN,NaN,NaN,NaN
76,158624819,2021-10-21T20:03:05.776Z,2021-10-21T20:03:06.305Z,SUCCESS,DEPOSIT,FIAT,30000,CLP,8771128318.00000000000,0cbfc255-2454-44e2-be92-d7f823347525,NaN,NaN,NaN,NaN


In [16]:
#I won't be loading my latest transactions, and for the demostrations the first 10 will be just as good
transactions_history = transactions_history.tail(10)

<h2>Streamlit Archives</h2>

In [17]:
currencies.to_csv('/Users/brauliocastelblancocastro/Desktop/currencies.csv', sep=',',index=False)

In [18]:
balance.to_csv('/Users/brauliocastelblancocastro/Desktop/balance.csv', sep=',',index=False)

In [19]:
transactions_history.to_csv('/Users/brauliocastelblancocastro/Desktop/transactions_history.csv', sep=',',index=False)